<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/notebooks/2020_0512covid19_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 武漢ウィルス covid-19 の動向を知りながら colab に慣れよう
- date: 2020-0518
- author: 浅川伸一
- source: https://towardsdatascience.com/say-goodbye-to-excel-a-simple-evaluation-of-python-grid-studio-using-covid-19-data-90624f322b81
- note: 駒澤大学 2020 年度教材

# まずは世界編

In [0]:
import numpy as np
import pandas as pd

In [0]:
# Read all data
df = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv").dropna()
print(df.head())

In [0]:
# Convert date to integer (because of Grid Studio limitation)
df.dateRep = pd.to_datetime(df.dateRep, format='%d/%m/%Y').dt.strftime('%Y%m%d').astype(int)

In [0]:
# Get Australia data
df_jp = df[df.countriesAndTerritories == 'Japan']
df_jp

In [0]:
#df_jp['cases'].plot()
import matplotlib.pyplot as plt

last = 30
d = [x for x in range(len(df_jp['dateRep'][:last]))]
plt.plot(df_jp['deaths'][:30])  # 死者数のプロット

In [0]:
df_jp['deaths'].sum()  # 死者数総計

In [0]:
df['deaths'].sum()  # 全世界の死者数総計

In [0]:
#df[df.countriesAndTerritories == 'Japan']['deaths'].sum()
df[df.countriesAndTerritories == 'Japan']['popData2018'][:1]

In [0]:
countries = set(sorted(df.countriesAndTerritories))
#print(countries)
for i, c in  enumerate(sorted(countries)):
    if (i+1) % 10  == 0:
        print('')
    print(c,end=" ")

In [0]:
countries = ['Japan', 'Taiwan', 'United_States_of_America', 'Spain', 'Italy']
for c in countries:
    print(c, df[df.countriesAndTerritories == c]['deaths'].sum(), df['popData2018'][0])

# ここから先は東京都のデータ

In [0]:
import json
import urllib.request

# source: https://github.com/tokyo-metropolitan-gov/covid19
tokyo_json_file = 'https://raw.githubusercontent.com/tokyo-metropolitan-gov/covid19/development/data/data.json'

req = urllib.request.Request(tokyo_json_file)
with urllib.request.urlopen(req) as response:
    data = json.load(response)

In [0]:
for k in data.keys():
    print(k)

In [0]:
print(data['lastUpdate'])
data['main_summary']

In [0]:
data['main_summary']['children']

In [0]:
data['patients_summary']['data'][-7:] # ['小計']
for x in data['patients_summary']['data'][-7:]:
    print(x['小計'])

In [0]:
patients_ = [x['小計'] for x in data['patients_summary']['data'][-28:]]
#plt.plot(patients_)
plt.bar(range(len(patients_)), patients_)
save_file_name = '2020-0518Tokyo_covid19_patients_summary.png'
plt.savefig(save_file_name)

In [0]:
# 自分の PC にグラフをダウンロードしてみましょう
from google.colab import files
files.download(save_file_name)

In [0]:
plt.plot(patients_)

In [0]:
print(data['inspection_persons']['datasets'][0]['label'])
print(data['inspection_persons']['datasets'][0]['data'])
plt.plot(data['inspection_persons']['datasets'][0]['data'][-21:])

In [0]:
data['main_summary']